In [1]:
from numpy import *

In [2]:
def loadExData():
    return [[1,1,1,0,0],
           [2,2,2,0,0],
           [1,1,1,0,0],
           [5,5,5,0,0],
           [1,1,0,2,2],
           [0,0,0,3,3],
           [0,0,0,1,1]]

In [3]:
Data = loadExData()
U, sigma, VT = linalg.svd(Data)
sigma

array([9.72140007e+00, 5.29397912e+00, 6.84226362e-01, 4.11502614e-16,
       1.36030206e-16])

后两个数值数量级太小，于是可以删掉

In [4]:
sig3 = mat([[sigma[0], 0, 0], [0, sigma[1], 0], [0, 0, sigma[2]]])
U[:, :3] * sig3 * VT[:3, :]  #这三个矩阵相乘就对原矩阵做了近似,形状不变

matrix([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          7.75989921e-16,  7.71587483e-16],
        [ 2.00000000e+00,  2.00000000e+00,  2.00000000e+00,
          3.00514919e-16,  2.77832253e-16],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          2.18975112e-16,  2.07633779e-16],
        [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00,
          3.00675663e-17, -1.28697294e-17],
        [ 1.00000000e+00,  1.00000000e+00, -5.48397422e-16,
          2.00000000e+00,  2.00000000e+00],
        [ 3.21319929e-16,  4.43562065e-16, -3.48967188e-16,
          3.00000000e+00,  3.00000000e+00],
        [ 9.71445147e-17,  1.45716772e-16, -1.52655666e-16,
          1.00000000e+00,  1.00000000e+00]])

#### 如何确定保留多少奇异值？  
 - 计算总能量信息，可以将奇异值求平方和，加到超过90%以上就可以   
 - 如果奇异值有上万个，就保留前2000或者3000个

# 基于协同过滤的推荐系统

### 相似度计算
 - 第一种方法：相似度 = 1 / （1 + 欧式距离）
 - 第二种方法：相似度 = 0.5 + 0.5 * corrcoef()   因为Pearson相关系数取值是在-1到+1之间
 - 第三种方法：相似度 = 0.5 + 0.5 * 余弦相似度

In [34]:
def ecludSim(inA, inB):
    return 1.0 / (1.0 + linalg.norm(inA - inB))

def pearsSim(inA, inB):
    if len(inA) < 3:
        return 1.0  # 如果向量长度小于3，那么计算出来的pearson相关系数必定是1，两个向量完全相关
    return 0.5 + 0.5 * corrcoef(inA, inB, rowvar=0)[0][1]

def cosSim(inA, inB):
    #print((inA.T * inB)[0][0])  ###
    return 0.5 + 0.5 * (float(inA.T * inB) / (linalg.norm(inA) * linalg.norm(inB)))

In [26]:
myMat = mat(Data)

In [27]:
print('>>>>>>>>>>>>>欧式距离相似度', euclidSim(myMat[:,0], myMat[:,4]))
print('>>>>>>>>>>>>>余弦相似度', cosSim(myMat[:,0], myMat[:,4]))
print('>>>>>>>>>>>>>pearson相关系数相似度', pearsSim(myMat[:,0], myMat[:,4]))

>>>>>>>>>>>>>欧式距离相似度 0.13367660240019172
>>>>>>>>>>>>>余弦相似度 0.5472455591261534
>>>>>>>>>>>>>pearson相关系数相似度 0.23768619407595815


如果用户数目多，就倾向于使用基于物品相似度的计算方法

In [28]:
def standEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]  #物品数目
    simTotal = 0.0
    ratSimTotal = 0.0
    for j in range(n):
        userRating = dataMat[user, j]
        if userRating == 0.0:  #评分为0，跳过这个物品
            continue
        ##这里给出的是符合条件的前面那个列向量里的元素的索引!
        overLap = nonzero(logical_and(dataMat[:, item].A>0, dataMat[:, j].A>0))[0]  
        if len(overLap) == 0.0:
            similarity = 0
        else:
            similarity = simMeas(dataMat[overLap, item], dataMat[overLap, j])
        print(f"the {item}th and {j}th's similarity is {similarity} ")
        simTotal += similarity
        ratSimTotal += similarity * userRating ##把该用户对每列物品的评分作为权重乘上去,结果会落在1-5之间
    if simTotal == 0.0:
        return 0
    else:
        return ratSimTotal / simTotal  ##对该item物品最终的预测值
    
    
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    unratedItems = nonzero(dataMat[user, :].A == 0)[1]  #该用户没预测的那些items的列索引
    if len(unratedItems) == 0.0:
        return 'you rated everything'
    itemScores = []
    for item in unratedItems:  #把每个没预测的item放到上面那个函数里，得到一个预测值
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
    return sorted(itemScores, key = lambda jj: jj[1], reverse= True)[:N]

In [29]:
myMat = mat(loadExData())

In [30]:
myMat[0,1] = myMat[0,0] = myMat[1,0] = myMat[2,0] = 4
myMat[3,3] = 2

In [31]:
myMat

matrix([[4, 4, 1, 0, 0],
        [4, 2, 2, 0, 0],
        [4, 1, 1, 0, 0],
        [5, 5, 5, 2, 0],
        [1, 1, 0, 2, 2],
        [0, 0, 0, 3, 3],
        [0, 0, 0, 1, 1]])

In [32]:
recommend(myMat, 2)

the 3th and 0th's similarity is 0.9160251471689218 
the 3th and 1th's similarity is 0.9160251471689218 
the 3th and 2th's similarity is 1.0 
the 4th and 0th's similarity is 1.0 
the 4th and 1th's similarity is 1.0 
the 4th and 2th's similarity is 0 


[(4, 2.5), (3, 1.9703483892927431)]

In [35]:
recommend(myMat, 2, simMeas=ecludSim)

the 3th and 0th's similarity is 0.2402530733520421 
the 3th and 1th's similarity is 0.2402530733520421 
the 3th and 2th's similarity is 0.25 
the 4th and 0th's similarity is 0.5 
the 4th and 1th's similarity is 0.5 
the 4th and 2th's similarity is 0 


[(4, 2.5), (3, 1.98665729687295)]

# 举例更加稀疏的矩阵

In [36]:
def loadExData2():
    return[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]

In [37]:
data = loadExData2()
U, sigma, VT = linalg.svd(data)

In [38]:
sigma

array([15.77075346, 11.40670395, 11.03044558,  4.84639758,  3.09292055,
        2.58097379,  1.00413543,  0.72817072,  0.43800353,  0.22082113,
        0.07367823])

### 看一下多少奇异值能达到总能量的90%

In [41]:
Sig2 = sigma ** 2
sum(Sig2)

541.9999999999995

In [42]:
sum(Sig2) * 0.9

487.7999999999996

In [43]:
sum(Sig2[:3])

500.5002891275793

### ↑说明前3个元素高于总能量的90%
<br/>
### ↓用于替换之前的standEst函数

In [48]:
def svdEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0
    ratSimTotal = 0.0
    U, sigma, VT = linalg.svd(dataMat)
    sig4 = mat(eye(4) * sigma[:4])    
    # ↑建立对角矩阵，↓把物品转换到低维空间中！行数不变，列数变成4
    xformedItems = dataMat.T * U[:, :4] * sig4.I
    for j in range(n):
        userRating = dataMat[user, j]
        if userRating == 0 or j == item:
            continue
        similarity = simMeas(xformedItems[item, :].T, xformedItems[j,:].T)  #注意这里的转置
        print(f'the {item} and {j}"s similarity is {similarity}')
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0:
        return 0
    else:
        return ratSimTotal / simTotal

In [49]:
data = mat(data)
recommend(data, 1, estMethod=svdEst)

the 0 and 3"s similarity is 0.4909504362809656
the 0 and 5"s similarity is 0.4842736086328837
the 0 and 10"s similarity is 0.5127549449063614
the 1 and 3"s similarity is 0.4912944647425579
the 1 and 5"s similarity is 0.48151631061179956
the 1 and 10"s similarity is 0.5097088624883724
the 2 and 3"s similarity is 0.49157334483312753
the 2 and 5"s similarity is 0.48234642287065527
the 2 and 10"s similarity is 0.5105839479452583
the 4 and 3"s similarity is 0.45049477333058713
the 4 and 5"s similarity is 0.5067948022540271
the 4 and 10"s similarity is 0.5128956287600113
the 6 and 3"s similarity is 0.7436990825417139
the 6 and 5"s similarity is 0.4683659603773957
the 6 and 10"s similarity is 0.43946465013002367
the 7 and 3"s similarity is 0.48217502131974066
the 7 and 5"s similarity is 0.4947163013621477
the 7 and 10"s similarity is 0.5249702098778073
the 8 and 3"s similarity is 0.49130720909511133
the 8 and 5"s similarity is 0.4912277621682814
the 8 and 10"s similarity is 0.5202895926889712

[(4, 3.344714938469228), (7, 3.3294020724526967), (9, 3.3281008763900686)]

## 看一下重构后的数据

In [52]:
dataMat = loadExData2()
dataMat = mat(dataMat)
U, sigma, VT = linalg.svd(dataMat)
sig4 = mat(eye(4) * sigma[:4])    
xformedItems = dataMat.T * U[:, :4] * sig4.I
print(xformedItems)

[[-0.45137416  0.03084799 -0.00290108  0.01189185]
 [-0.36239706  0.02584428 -0.00189127  0.01348796]
 [-0.46879252  0.03296133 -0.00281253  0.01656192]
 [-0.01007685 -0.34024331 -0.22728592  0.14546051]
 [-0.01567036 -0.38750193  0.61197998 -0.17137451]
 [-0.01664563 -0.52000097 -0.3608907  -0.14984063]
 [-0.00474684 -0.18887149 -0.00924222  0.94228361]
 [-0.46712774  0.00389831  0.03349951 -0.02080674]
 [-0.47223188  0.02853952 -0.00504059  0.00160266]
 [-0.01591788 -0.39205093  0.55707516  0.04356321]
 [-0.0552444  -0.52034959 -0.36330956 -0.19023805]]


### ↑发现行数没变，列数变成4了

## 以上推荐引擎存在的问题
 - SVD会降低程序的速度，可以程序调用时运行一下或者离线运行
 - 可以存储非零元素节省空间，相似度计算造成计算资源浪费，可以离线运行
 - 冷启动问题，可以将推荐看成是搜索问题，需要所推荐物品的属性，这些属性可以作为相似度计算所需要的数据，这被称为基于内容的推荐

# 示例： 基于SVD的图像压缩

In [66]:
def printMat(inMat, thresh=0.8):
    for i in range(32):
        for k in range(32):
            if float(inMat[i, k]) > thresh:
                print  (1, end='')
            else:
                print(0, end='')
        print('')
        
def imgCompress(numSV=3, thresh=0.8):
    myl = []
    for line in open('0_5.txt').readlines():
        newRow = []
        for i in range(32):
            newRow.append(int(line[i]))
        myl.append(newRow)
    myMat = mat(myl)
    print('********************原始矩阵*********************')
    printMat(myMat, thresh)
    U, sigma, VT = linalg.svd(myMat)
    sigRecon = mat(zeros((numSV, numSV)))
    for k in range(numSV):
        sigRecon[k,k] = sigma[k]
    reconMat = U[:, :numSV] * sigRecon * VT[:numSV, :]
    print(f'*****************用{numSV}个奇异值重构后的矩阵*********************')
    printMat(reconMat, thresh)

### 直接打印矩阵和用画图函数的区别↓

In [67]:
myl = []
for line in open('0_5.txt').readlines():
    newRow = []
    for i in range(32):
        newRow.append(int(line[i]))
    myl.append(newRow)
myMat = mat(myl)
print(myMat)

printMat(myMat, thresh=0.8)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
00000000000000110000000000000000
00000000000011111100000000000000
00000000000111111110000000000000
00000000001111111111000000000000
00000000111111111111100000000000
00000001111111111111110000000000
00000000111111111111111000000000
00000000111111100001111100000000
00000001111111000001111100000000
00000011111100000000111100000000
00000011111100000000111110000000
00000011111100000000011110000000
00000011111100000000011110000000
00000001111110000000001111000000
00000011111110000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000001111100000000011111000000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000011111000000
00000000111110000000111111000000
00000000111111000001111110000000
0000000001111111111111

In [68]:
imgCompress(2)

********************原始矩阵*********************
00000000000000110000000000000000
00000000000011111100000000000000
00000000000111111110000000000000
00000000001111111111000000000000
00000000111111111111100000000000
00000001111111111111110000000000
00000000111111111111111000000000
00000000111111100001111100000000
00000001111111000001111100000000
00000011111100000000111100000000
00000011111100000000111110000000
00000011111100000000011110000000
00000011111100000000011110000000
00000001111110000000001111000000
00000011111110000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000001111100000000011111000000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000011111000000
00000000111110000000111111000000
00000000111111000001111110000000
00000000011111111111111110000000
00000000001111111111111110000000
000000000011111111111111100000

## ↑可以看到，只需两个奇异值就能精准的对图像进行重构，数据量进行了极大的压缩
 - 原始数据 $ 32*32 = 1024 $
 - 重构后的数据 $ 32*2 + 32*2 + 2 = 130 $